In [1]:
import os
import glob
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np

In [2]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

In [3]:
import os

# Suppress TensorFlow Python logs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Suppress ML Compute / Metal debug info
os.environ["TF_METAL_ENABLE_LOGGING"] = "0"
os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"

import tensorflow as tf
tf.debugging.set_log_device_placement(False)


In [4]:
# Dataset setup and exploration
dataset_path = "/Volumes/CrucialX9/Project/Data Science/Image Based/dataset"

In [ ]:
import os
import glob
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np

# Mixed precision for faster training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

# Suppress TensorFlow logs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TF_METAL_ENABLE_LOGGING"] = "0"
os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"
tf.debugging.set_log_device_placement(False)

# Dataset setup
dataset_path = "/Volumes/CrucialX9/Project/Data Science/Image Based/dataset"
classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
print("Found classes:", classes)
print("Total classes:", len(classes))

# Count images in each class
for cls in classes:
    class_path = os.path.join(dataset_path, cls)
    image_files = glob.glob(os.path.join(class_path, "*.[pjPJ]*[ngNG]*"))  # jpg, jpeg, png
    print(f"Class '{cls}' has {len(image_files)} images")

# Image parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20  # Reduce epochs to speed up training
VAL_SPLIT = 0.15
SEED = 1337

# Load the dataset
train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
)

val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
)

# Data augmentation and preprocessing
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
])

# Prefetch for performance optimization
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# Load pre-trained EfficientNetB3
base_model = tf.keras.applications.EfficientNetB3(
    include_top=False,
    weights="/Volumes/CrucialX9/Project/Data Science/Image Based/TEST/efficientnetb3_notop.h5",  # Load pre-trained weights from ImageNet
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling="avg",  # Global average pooling
)

# Unfreeze some layers for fine-tuning
base_model.trainable = True
fine_tune_at = 100  # Fine-tune from the 100th layer onward
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Build the model
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet.preprocess_input(x)
x = base_model(x, training=False)
x = layers.Dropout(0.3)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(classes), activation="softmax")(x)

model = models.Model(inputs, outputs)

# Compile the model with an initial learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

# Callbacks
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),  # Early stopping with patience
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2),  # Reduce learning rate when plateau is reached
    LearningRateScheduler(lambda epoch: 1e-5 * 10 ** (epoch / 10)),  # Learning rate scheduler
]

# Initial training
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
)

# Fine-tuning: Unfreeze more layers and continue training
base_model.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

fine_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,  # Fine-tune for fewer epochs (5-10 epochs)
    callbacks=callbacks,
)

# Save the model
model.save("skin_disease_model_tf.h5")
model.save("skin_disease_model_tf.keras")


Found classes: ['Acne And Rosacea Photos', 'Malignant', 'Fu Athlete Foot', 'Eczema Photos', 'Fu Ringworm', 'Benign', 'Exanthems And Drug Eruptions', 'Lupus And Other Connective Tissue Diseases', 'Ba Impetigo', 'Bullous Disease Photos', 'Nail Fungus And Other Nail Disease', 'Light Diseases And Disorders Of Pigmentation', 'Atopic Dermatitis Photos', 'Actinic Keratosis Basal Cell Carcinoma And Other Malignant Lesions', 'Melanoma Skin Cancer Nevi And Moles', 'Cellulitis Impetigo And Other Bacterial Infections', 'Hair Loss Photos Alopecia And Other Hair Diseases', 'Herpes Hpv And Other Stds Photos', 'Fu Nail Fungus', 'Ba  Cellulitis']
Total classes: 20
Class 'Acne And Rosacea Photos' has 6837 images
Class 'Malignant' has 6762 images
Class 'Fu Athlete Foot' has 8054 images
Class 'Eczema Photos' has 6715 images
Class 'Fu Ringworm' has 8129 images
Class 'Benign' has 6459 images
Class 'Exanthems And Drug Eruptions' has 7750 images
Class 'Lupus And Other Connective Tissue Diseases' has 7734 imag